In [2]:
import tensorflow as tf
tf.__version__

'2.7.0'

### Sequential API

In [3]:
input_dim = (32, 32, 3)
output_dim = 10
seq_model = tf.keras.Sequential()
seq_model.add(tf.keras.Input(shape=input_dim))

# Block 1
seq_model.add(tf.keras.layers.Conv2D(32, 3, activation=tf.keras.activations.relu))
seq_model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
seq_model.add(tf.keras.layers.BatchNormalization())

# Block 2
seq_model.add(tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu))
seq_model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
seq_model.add(tf.keras.layers.BatchNormalization())
seq_model.add(tf.keras.layers.Dropout(0.2))

seq_model.add(tf.keras.layers.GlobalMaxPool2D())

seq_model.add(tf.keras.layers.Dense(output_dim))

### Functional API

In [4]:
input = tf.keras.Input(shape=input_dim)

# Block 1
x = tf.keras.layers.Conv2D(32, 3, activation=tf.keras.activations.relu)(input)
x = tf.keras.layers.MaxPool2D(2, strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

# Block 2
x = tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.MaxPool2D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.GlobalMaxPooling2D()(x)

output = tf.keras.layers.Dense(output_dim)(x)

functional_api_model = tf.keras.Model(input, output)

### Model Subclassing API

In [5]:
class SubclassedModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(SubclassedModel, self).__init__()

        # Block 1 layers
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation=tf.keras.activations.relu)
        self.mp2d_1 = tf.keras.layers.MaxPool2D(2)
        self.bn1 = tf.keras.layers.BatchNormalization()

        # Block 2 layers
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu)
        self.mp2d_2 = tf.keras.layers.MaxPool2D(2)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.dropout1 = tf.keras.layers.Dropout(0.2)

        self.gmp2d_1 = tf.keras.layers.GlobalMaxPool2D()
        self.dense = tf.keras.layers.Dense(num_classes)
    
    def call(self, input_tensor,training=False):
        # Block 1 forward prop
        x = self.conv1(input_tensor)
        x = self.mp2d_1(x)
        x = self.bn1(x)

        # Block 2 Forward prop
        x = self.conv2(x)
        x = self.mp2d_2(x)
        x = self.bn2(x)
        x = self.dropout1(x)

        # GMP and classifier
        x = self.gmp2d_1(x)
        x = self.dense(x)

        return x

subclassed_model = SubclassedModel(10)

### Inception using subclassing

In [6]:
class ConvModule(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1,1), padding='valid'):
        super(ConvModule, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding
        )

        self.bn1 = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = tf.keras.layers.ReLU()(x)

        return x

conv_module_test = ConvModule(32, (3,3), (2,2), padding='same')
y = conv_module_test(tf.random.normal(shape=(5,32,32,3)))

print("Weights:", len(conv_module_test.weights))
print("Trainable weights:", conv_module_test.trainable_weights)

Weights: 6
Trainable weights: [<tf.Variable 'conv_module/conv2d_6/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
array([[[[ 0.09819677,  0.04250462, -0.03227864,  0.08331011,
          -0.01920497, -0.08719671, -0.06369919, -0.03679688,
          -0.12237141,  0.11439265, -0.09071819,  0.01125498,
           0.09286787,  0.01422735,  0.08453463,  0.04890171,
           0.04485255, -0.02455962, -0.04500244,  0.0379802 ,
          -0.02382643,  0.00198989, -0.00914401,  0.10190213,
           0.02109134, -0.07022265,  0.08869596,  0.03815177,
          -0.09017881, -0.12084728, -0.11545612,  0.00068988],
         [-0.13052982, -0.1075158 , -0.07580997,  0.1028346 ,
           0.00998785, -0.03325227, -0.07679123,  0.01074548,
           0.03177842, -0.05734852,  0.1332571 ,  0.00150441,
           0.06180944, -0.11915239,  0.12868981,  0.02851692,
          -0.03821439, -0.12645501, -0.12364648,  0.07344055,
          -0.12966405,  0.05714901,  0.12059675,  0.06019133,
          -0.

In [15]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3):
        super(InceptionModule, self).__init__()

        self.conv1 = ConvModule(filters_1x1, kernel_size=(1,1), strides=(1,1), padding='same')
        self.conv2 = ConvModule(filters_3x3, kernel_size=(3,3), strides=(1,1), padding='same')
        
    def call(self, input_tensor, training=False):
        x_1x1 = self.conv1(input_tensor)
        x_3x3 = self.conv2(input_tensor)
        x = tf.keras.layers.Concatenate([x_1x1, x_3x3])
        
        return x

inception_module_test = InceptionModule(32, 32)
y = inception_module_test(tf.random.normal(shape=(5, 32, 32, 3)))
print("Weights:", y.weights)

Weights: []


In [17]:
class DownsampleModule(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(DownsampleModule, self).__init__()
        self.conv1 = ConvModule(
            filters=filters,
            kernel_size=(3,3),
            strides=(2,2),
            padding='valid'
        )

        self.mp2d_1 = tf.keras.layers.MaxPool2D((3,3), strides=(2,2))
    
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        y = self.mp2d_1(input_tensor)
        
        return tf.keras.layers.Concatenate([x, y])

downsample_module_test = DownsampleModule(32)
y = downsample_module_test(tf.random.normal(shape=(5,32,32,3)))
print(y)